# Homework 6

Author: Sherry

# Exercise 2

## (a) Write a code that prices, through backward induction, a European Call which payoff is given by (SN-K)+ for K = 60.

S0=50, time interval=1 day, u=1.01, d=0.99, r=0.01/126 per day, N=504 days, K=60.

In [1]:
#Define the StockPrice() function to calculate the stock price at node (N,i)
#i is the number of steps the stock price goes up, (N-i) is the number of 
#steps the stock price goes down.
def StockPrice(S0,u,d,N,i):
    return S0*pow(u,i)*pow(d,N-i)


#Define the CallPatoff() to calculate the call option payoff
def CallPayoff(St,K):
    if St>K:
        return St-K
    else:
        return 0

#Define the Function1().
#It computes the (N!)/(i!(N-i)!) term which will be used in the CallPrice() 
#function.
def Function1(N,i):
    result=1
    n=1
    while n<=i:
        result=result*(N-i+n)/n
        n=n+1
    return result

#Define the CallPrice() to compute the call option price at time t=0
def CallPrice(S0,u,d,r,N,K):
    #compute the risk-neutral probability
    p=(1+r-d)/(u-d)
    
    #compute the call option price
    price=0
    i=0
    while i<=N:
        price=price+Function1(N,i)*pow(p,i)*pow(1-p,N-i)*\
        CallPayoff(StockPrice(S0,u,d,N,i),K)/pow(1+r,N)
        i=i+1
    return price

#Define the Delta() to compute call option Delta at time t=0
def Delta(S0,u,d,r,N,K):
    #compute the stock prices and call option prices at time t=1
    Su=S0*u
    Sd=S0*d
    price_u=CallPrice(Su,u,d,r,N-1,K)
    price_d=CallPrice(Sd,u,d,r,N-1,K)
    
    #compute Delta at time t=0
    delta=(price_u-price_d)/(Su-Sd)
    return delta
    

#test
S0=50
u=1.01
d=0.99
r=0.01/126
N=504
K=60
C=CallPrice(S0,u,d,r,N,K)
Delta=Delta(S0,u,d,r,N,K)
print("European call option price = ",C)
print("Delta = ",Delta)


European call option price =  1.917161602552663
Delta =  0.30078185398544566


## (b) Write a code that prices, through backward induction, a Lookback option which payoff is given by max_{0≤n≤N}(SN-Sn).

S0=50, time interval=1 day, u=1.01, d=0.99, r=0.01/126 per day, N=20 days.

In [2]:
import numpy as np

#Define the LookbackPrice() to compute the lookback option price at time t=0
def LookbackPrice(S0,u,d,r,N):
    #compute the risk-neutral probability
    p=(1+r-d)/(u-d)
    
    #simulate the path (there are 2^N paths) and store the paths in a
    path=[]
    a=np.zeros(shape=(pow(2,N),N+1))
    down=[]
    for i in range(0,pow(2,N)):
        path.append(bin(i).zfill(N)) 
    #here we use the binary format of the integers from 0 to (2^N)-1
    #to simulate the path: with the binary format of each integer,
    #a digit of 1 represents a down step and a digit of 0 represents
    #an up step. Since all the 2^N integers can be expressed in 10-bit
    #binary format, there are 2^N different paths in total.
    
    for i in range(0,pow(2,N)):
        c=0  #count the number of down steps
        S=S0
        a[i][0]=S #the first column of a is the stock price at time time=0
        j=-1 
        while j>=-N:  #change the binary digits to the stock price
            if (str(path[i][j])=="1"): #when there is a "1", the stock price
                                       #goes down
                a[i][-j]=S*d
                c=c+1
            else:  #when there is a "0", the stock price goes up
                a[i][-j]=S*u
            S=a[i][-j]
            j=j-1
        down.append(c)
            
    #compute the payoff of each path and store the payoffs in b
    b=[] 
    for i in range(0,pow(2,N)):
        b.append(a[i][N]-min(a[i])) #the payoff is S_T minus the lowest stock
                                    #price in the path
        
    #compute the price
    price=0
    for k in range(0,pow(2,N)):
        price=price+b[k]*pow(p,N-down[k])*pow(1-p,down[k])/pow(1+r,N)
    return price

def LookbackDelta(S0,u,d,r,N):
    #compute the stock prices and option prices at time t=1
    Su=S0*u
    Sd=S0*d
    price_u=LookbackPrice(Su,u,d,r,N-1)
    price_d=LookbackPrice(Sd,u,d,r,N-1)
    
    #compute Delta at time t=0
    delta=(price_u-price_d)/(Su-Sd)
    return delta


#test
S0=50
u=1.01
d=0.99
r=0.01/126
N=20

Price=LookbackPrice(S0,u,d,r,N)
LookbackDelta=LookbackDelta(S0,u,d,r,N)
print("Lookback option price = ",Price)
print("Delta = ",LookbackDelta)


Lookback option price =  1.58079109388
Delta =  0.0307128870843


## (c) Write a code that prices, through backward induction, an American call option with strike K = 60, which is a European call option that you can choose to exercise at any time between 1, ... , N.

In [3]:
import numpy as np

#Define the function AmCallPrice() to compute the American call option price 
#at time t=0
def AmCallPrice(S0,u,d,r,N,K):
    #compute the risk-neutral probability
    p=(1+r-d)/(u-d)
    Tree = np.zeros(shape=(N+1,N+1))
    for i in range(0,N+1): #compute the payoffs at time t=T
        Tree[N][i]=CallPayoff(StockPrice(S0,u,d,N,i),K)
    n=N-1
    while n>=0: 
        for i in range(0,n+1):
            #compute the payoff through backward induction
            value=(p*Tree[n+1][i+1]+(1-p)*Tree[n+1][i])/(1+r) 
            #compute the payoff if we exercise the option at the current node
            Tree[n][i]=CallPayoff(StockPrice(S0,u,d,n,i),K)
            #compare the two payoffs to decide if we exercise the option at 
            #this node
            if (value>Tree[n][i]):
                Tree[n][i]=value
        n=n-1
    return Tree[0][0]

#Test
S0=50
u=1.01
d=0.99
r=0.01/126
N=504
K=60
a=AmCallPrice(S0,u,d,r,N,K)
print("American call option price = ",a)


American call option price =  1.91716160255
